In [107]:
import numpy as np
import textdistance
import timeit
import pandas as pds
from sklearn import mixture
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [108]:
identifiers = {'family name':'jaro-winkler','gender':'strict','country':'strict','birth year':'large'}
covariates = ['X1','X2','X3','X4','X5']

name_DF = 'DF_associations_N=73_2023-01-04.csv'
DF = pds.read_csv(os.path.join('..','..','simulate data','datasets', name_DF), delimiter = ',')

overlap = DF.sample(n = 15)

A = pds.concat([DF.sample(n = 45), overlap]).drop(['Y'], axis = 1)
A = A.reset_index(drop=True)

B = pds.concat([DF.sample(n = 35), overlap])[list(identifiers.keys())+['Y']]
B = B.reset_index(drop=True)

In [109]:
A.head()

,name,family name,gender,country,birth year,treatment,X1,X2,X3,X4,X5
0,Nina,Yashina,F,RU,1960,0,60,8.089946,4.279396,8.839126,6.761503
1,Dirk-Bijan,Zarrinnam,M,DE,1981,0,39,8.876769,2.346136,8.148365,4.681018
2,Michelle,Brigden,F,GB,1989,0,31,8.133766,1.856616,8.478746,5.266938
3,Christin,Müller,F,DE,1965,0,55,8.746727,1.668986,8.795278,7.700038
4,Diabolik,Preti,F,IT,1989,0,31,8.231791,0.500859,9.250222,5.189169


In [110]:
B.head()

,family name,gender,country,birth year,Y
0,Raj,M,FR,1979,70.794479
1,Ghinassi,M,IT,1945,34.448177
2,Perez Simon,M,ES,1987,78.687278
3,Preti,F,IT,1989,7.369748
4,Zaworski,M,PL,2019,-10.102921


In [111]:
def levenshtein_similarity(a,b):

    """ Check that levenshtein similarity (in [0,1]) is above 0.95.
        
        a: string,
        b: string """

    if 1 - textdistance.levenshtein(a, b)/max(len(a),len(b)) >= 0.95:
        return 1
    else:
        return 0

def jaro_winkler_similarity(a,b):

    """ Check that jaro-winkler similarity (in [0,1]) is above 0.95.
        
        a: string,
        b: string """

    if textdistance.jaro_winkler(a,b) >= 0.99:
        return 1
    else:
        return 0

def strict_equality(a,b):

    """ Check that a and b values are equal.
        
        a: any value,
        b: any value """

    return a==b

def large_equality(a,b):

    """ Check that years a and b expressed with four numbers are within the same decade.
        
        a: year,
        b: year """

    return str(a)[:-1]==str(b)[:-1]

In [112]:
def comparison_vector(A_record, B, identifiers):
    
    """ Compare one record in A with all records in B. 
        Return the binary comparison of the identifiers for one record in A with all records in B.

        A_record:     series of one row, 
        B:            dataframe, 
        identifiers:  dict: k = column name, 
                            v = method in {'large','strict','levenshtein','jaro-winkler'}
    """

    methods = {'jaro-winkler':jaro_winkler_similarity, 'levenshtein':levenshtein_similarity, 'strict':strict_equality, 'large':large_equality}
    comparisons = {}
    for linking_var in identifiers:
        method = methods[identifiers[linking_var]]
        comparisons[linking_var] = np.array(B.apply(lambda row: method(A_record[linking_var], row[linking_var]), axis=1)).reshape(-1,1)
    return np.concatenate(tuple(comparisons.values()), axis = 1) 
    
A_record = A.iloc[40,:]
comparison_vector(A_record, B, identifiers)

array([[0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 1],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 1,

In [114]:
# Parameters
# match: probability of having same linking var when being true matches
# unmatch: probability of having same linking var (at all)

match = np.repeat(0.95, len(identifiers.keys()))
unmatch = A.apply(lambda row: comparison_vector(row, B, identifiers).sum(axis=0), axis=1).sum() / (A.shape[0]*B.shape[0]) # for each A record and for each linking variable, we check 'the probability' to match (over the nA * nB pairs)

In [115]:
def linking_score(A, B, identifiers, match, unmatch):
        
    """ Compare records in A with records in B, computing all linking scores for records in A with records in B. 
        Return the indices of records in A with the best match index for record in B.

        A:            dataframe, 
        B:            dataframe, 
        identifiers:  dict: k = column name, 
                            v = method in {'large','strict','levenshtein','jaro-winkler'}
        match:        array of probabilities of having same linking variables when being a match,
        unmatch:      array of probabilities of having same linking variables (at all, among the nA x nB pairs of record).
    """

    def compute_max_linking_score(A_record, B, identifiers, match, unmatch):
        similarities = comparison_vector(A_record, B, identifiers)
        if similarities.all(axis=1).sum() == 1: # there is one unique match: possible true match
            return B.iloc[similarities.all(axis=1),:].index[0], np.NaN
        else:
            linking_score = (np.multiply(similarities, np.log2(match/unmatch)) + np.multiply(1-similarities, np.log2((1-match)/(1-unmatch)))).sum(axis=1)
            return linking_score.argmax(), linking_score.max()

    links = A.apply(lambda row: compute_max_linking_score(row[list(identifiers.keys())], B, identifiers, match, unmatch), axis=1)
    idx_in_A = np.arange(A.shape[0])
    idx_in_B = np.array([element[0] for element in links])
    matching_scores = np.array([element[1] for element in links])
    max_score = np.nanmax(matching_scores)
    matching_scores = np.nan_to_num(matching_scores, nan=max_score+1) # ossible true matches are given a higher score
    return {'A':idx_in_A, 'B':idx_in_B, 'scores':matching_scores}

linking_score(A, B, identifiers, match, unmatch)

{'A': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59]),
 'B': array([20, 21, 18, 25,  3,  1,  9, 13, 24,  5, 37, 26,  6, 12, 43, 33,  1,
         6, 26, 12, 10, 48,  3, 48, 19, 28, 21, 24, 27, 22, 36, 32,  7, 15,
        25, 31,  4,  2, 26, 36, 11, 23,  8, 25,  0, 11, 36, 37,  3, 20, 40,
         7, 23, 43,  4, 31, 46, 47, 48, 49]),
 'scores': array([11.90991281,  2.02253727, 12.90991281, 12.90991281, 11.90991281,
         2.02253727, 12.90991281, 12.90991281, 12.90991281, -4.48586935,
        12.90991281,  2.02253727,  2.02253727,  2.02253727, 12.90991281,
        12.90991281, 12.90991281, 12.90991281, 12.90991281, 12.90991281,
        12.90991281, 12.90991281,  2.02253727, -2.24077913, 12.90991281,
        -2.24077913, 12.90991281,  2.02253727, -4.48586935, 12.9099

In [116]:
def stratified_ATE(DF_group, pop_size):

    """ Compute the Average Treatment Effect on the specific stratum represented in DF_group.

        DF_group: dataframe. 
    """

    n_treated, n_untreated = DF_group[DF_group.treatment == 1].shape[0], DF_group[DF_group.treatment == 0].shape[0]
    assert (n_treated!=0)&(n_untreated!=0), ("One group among treated/untreated is empty with this stratification.")
    avg_outcome_treated = (DF_group.treatment * DF_group.Y).sum() / n_treated 
    avg_outcome_untreated = ((1 - DF_group.treatment) * DF_group.Y).sum() / n_untreated
    var_treated = ( (DF_group.treatment * DF_group.Y - avg_outcome_treated)**2 ).sum() / (n_treated - 1)
    var_untreated = ( ((1 - DF_group.treatment) * DF_group.Y - avg_outcome_untreated)**2 ).sum() / (n_untreated - 1)
    size_group = DF_group.shape[0]
    ATE_group = (size_group/pop_size) * (avg_outcome_treated - avg_outcome_untreated)
    variance_group = (size_group/pop_size)**2 * (var_untreated/n_untreated + var_treated/n_treated)
    return ATE_group, variance_group

stratified_ATE(DF, DF.shape[0])

(32.294127882051875, 659.3656778153706)

In [117]:
def logit(p):
    return np.log(p/(1-p))

def propensity_score(DF, covariates, scaler, convert_to_logit):
    
    """ Compute propensity score estimates: the probability (logistic regression) that an observation is treated or not conditioned on some covariates.

        DF:                dataframe,
        covariates:        list of strings for covariates variable in DF,
        scaler:            sklearn.preprocessing function scaler for exemple,
        convert_to_logit:  boolean for converting probabilities to logit when building the propensity score estimates based on a logistic regression
    """

    if scaler != None:
        pipe = Pipeline([('scaler', scaler),('logistic_classifier', LogisticRegression())])
    else:
        pipe = Pipeline([('logistic_classifier', LogisticRegression())])
    pipe.fit(DF[covariates], DF.treatment)
    predictions = pipe.predict_proba(DF[covariates])
    if convert_to_logit:
        predictions_logit = logit(predictions[:,1])
        return predictions_logit
    else: 
        return predictions[:,1]

propensity_score(DF, covariates, MinMaxScaler(), True)

array([-0.24562854, -0.62480721, -0.83119896, -0.34601315, -0.56723824,
       -0.38506722, -0.42154433, -0.09940014, -0.75055998, -1.05732297,
       -0.96563948, -0.69103408, -0.62744196, -0.69477582, -0.79408168,
       -0.94897301, -1.06338381, -0.4150786 , -0.67903307, -0.67575906,
       -0.86799301, -0.69891275, -0.60579753, -0.68251425, -0.83952037,
       -0.54761415, -0.25637736, -0.42153576, -0.36500929, -0.56421259,
       -1.09662174, -0.68028281, -0.19789087, -0.38843211, -0.58636426,
        0.05849329, -0.68959717, -0.65863703, -0.77049132, -0.64023906,
       -0.47072301, -0.1497882 , -0.88817401, -0.54896316, -0.33271651,
       -0.04997487, -0.23080071, -0.56559951, -0.89318468, -0.37636724,
       -0.11269199, -0.36144691, -0.25485061, -0.29643366, -0.41099934,
       -0.59814537, -0.64360456, -0.46011144, -0.55477808, -0.4015365 ,
       -0.23718291, -0.51811752, -0.65835297, -0.62057378, -0.90680327,
       -0.14491584, -0.57845571, -0.68704653, -0.50665226, -0.17

In [119]:
def ATE(DF, strata): # covariates=None, scaler=None, convert_to_logit=None,
    
    """ Compute the Average Treatment Effect in DF according to the stratification method:
        no stratification when strata is None, 
        stratified dataframe build based on the list of specific covariates when one is passed,
        or propensity score stratification.
        Propensity score estimates are built conditionaly on covariates passed using a logistic regression after transformation by scaler (when one is specified).
        Estimated probabilities can be converted into logit (convert_to_logit parameter).
        Quantiles are used to partition the data based on propensity score estimates.
        'covariates', 'scaler', 'convert_to_logit', 'quantiles' are only needed for 'propensity stratification' method.

        DF:                dataframe,
        strata:            value in {None, [...], 'propensity stratification'},
        covariates:        list of strings for covariates variable in DF,
        scaler:            sklearn.preprocessing function scaler for exemple,
        convert_to_logit:  boolean for converting probabilities to logit when building the propensity score estimates based on a logistic regression,
        quantiles:         list of quantiles (>0) to consider to build strata based on propensity score
    """
    
    pop_size = DF.shape[0]
    if strata == None: # no stratification
        return stratified_ATE(DF, pop_size)
    elif strata == 'propensity stratification': # propensity score stratification
        assert ('propensity_score' in DF.columns)&('prop_score_quantile' in DF.columns), ("For propensity score stratification you need first to add the propensity_score and prop_score_quantile columns into the dataframe.")
        ATE, variance = 0, 0
        for q in DF['prop_score_quantile'].unique():
            stratum_data = DF[DF['prop_score_quantile'] == q]
            ATE_stratum, variance_stratum = stratified_ATE(stratum_data, pop_size)
            ATE += ATE_stratum
            variance += variance_stratum
        return ATE, variance
    else: # stratification based on the covariates passed
        ATE, variance = 0, 0
        for stratum in DF.groupby(strata):
            stratum_id, stratum_data = stratum
            ATE_stratum, variance_stratum = stratified_ATE(stratum_data, pop_size)
            ATE += ATE_stratum
            variance += variance_stratum
        return ATE, variance

ATE(DF, 'propensity stratification')

(19.206827174354896, 3387.381045181751)

In [120]:
ATE(DF, None)

(32.294127882051875, 659.3656778153706)

In [121]:
ATE(DF, ['gender'])

/var/folders/_7/32t15g3d2h54d0ct6cssb9gh0000gn/T/ipykernel_31775/3199416953.py:34: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for stratum in DF.groupby(strata):


(32.76974768988275, 706.732593652973)

In [17]:
def Estimate_Tethered_Stopping_Rule(A, B, identifiers, match, unmatch, covariates, scaler, convert_to_logit):

    """ Compare records in A with records in B, computing all linking scores for records in A with records in B. 
        Return the indices of records in A with the best match index for record in B.

        A: dataframe, 
        B: dataframe, 
        identifiers: dict: k = column name, v = method in {'large','strict','levenshtein','jaro-winkler',
        match: array of probabilities of having same linking variables when being a match,
        unmatch: array of probabilities of having same linking variables (at all, among the nA x nB pairs of record),
        strata: .
    """
    
    matchings = linking_score(A, B, identifiers, match, unmatch)
    for score in np.sort(np.unique(matchings['scores']))[::-1]:
        best_matches = matchings['A'][matchings['scores'] == score]
        from_A = A.iloc[best_matches,:].drop('source', axis=1).reset_index(drop=True)
        from_B = B.iloc[matchings['B'][best_matches],:]['Y'].reset_index(drop=True)
        linked_records = pds.concat([linked_records, pds.concat([from_A,from_B], axis = 1)])
        ATE_links, variance_links = ATE(linked_records, 'propensity stratification')
        ATE_list.append(ATE_links)
        Var_list.append(variance_links)
    return ATE_list, Var_list

A['propensity_score'] = propensity_score(A, covariates, MinMaxScaler(), True)
q = 5 # recommended
A['prop_score_quantile'] = pds.qcut(A['propensity_score'], q, labels = False)

# check the balance
np.array([(A[A['prop_score_quantile']==value].treatment.value_counts() > A.shape[0]/(q*3)).all() for value in range(q)]).all()

Estimate_Tethered_Stopping_Rule(A, B, identifiers, match, unmatch, ['X1','X2','X3'], MinMaxScaler(), True)

([47.33719513914916,
  46.39276952192214,
  46.153054900522065,
  45.27140935392173,
  44.72825817031678,
  15.600965590480033,
  15.49195327496395,
  15.309637357517015,
  13.781570614273592,
  13.781329335788602],
 [69.30381736813325,
  65.3316786061727,
  67.11109893024654,
  64.5648768468791,
  61.89730849861741,
  13.821247094960613,
  13.805832166752769,
  13.432986717798697,
  11.510034145134389,
  11.402201028704862])